# Computing confidence intervals for linear regression coefficients 

Assume we have a linear regression problem, where the data comes from the model

$$
y = \langle \textbf{w}, \textbf{x} \rangle + \phi
$$

where $\textbf{w} \in \mathbf{R}^2$ and $\phi \sim{\cal N}(0,\sigma^2)$.

#### Part 1. Assume $\sigma$ known and fixed. Thus, we estimate linear regression coefficients using normal distribution. Let's see an example. 

In [1]:
import numpy as np
from scipy.stats import t
np.random.seed(40)

m=150
d=1 #thus only one feature

#standard deviation of "noise" in the feature
sigma_x = 1.
#standard deviation of error
sigma_e = 0.1

#values on which depend the feature 
X_0 = np.ones((m,1))

#values of the feature = X_0 + noise (with standard deviation = sigma_x)
X_1 = X_0 + np.random.normal(loc=0.0, scale=sigma_x, size=(m,1))

#design matrix
X=np.hstack((X_0,X_1))

w = np.array([[3],[0]])
#note that the feature is not used in the model!

#Y = linear model as above (noise has standard devatiation = sigma_e)
Y = np.dot(X,w)+ np.random.normal(loc=0.0, scale=sigma_e, size=(m,1))

#compute the inverse of X^T dot X
XX_inv = np.linalg.inv(np.dot(np.transpose(X),X))

#compute the estimated weights of the linear model
w_hat = np.dot(XX_inv,np.dot(np.transpose(X),Y))

#compute the variance of the normal distribution from which the estimated weights come from 
var_w = np.array(np.diag(XX_inv) * sigma_e**2).reshape(2,1)

#compute the confidence intervals (using the normal distribution, assuming sigma_e known)
CI_min = w_hat - 1.96*np.sqrt(var_w) #alpha=0.05 
CI_max = w_hat + 1.96*np.sqrt(var_w) #alpha=0.05

print("Estimated weights of the linear model:\n", w_hat)
print("\nConfidence intervals for the two weights:\n",np.hstack((CI_min,CI_max)))

Estimated weights of the linear model:
 [[2.99641392e+00]
 [1.36817691e-03]]

Confidence intervals for the two weights:
 [[ 2.97410492  3.01872293]
 [-0.01505021  0.01778657]]


#### Part 2. Assume $\sigma$ unknown and to be estimated. Thus, we estimate linear regression coefficients using t-student distribution. Let's see an example. 

In [2]:
#compute estimated targets
y_hat = np.dot(X,w_hat)

#compute estimated variance sigma_e
sigma2_hat = (1./(m-d-1)) * sum((y_hat-Y)**2)
print("Estimated variance of noise:\n", sigma2_hat) #note: not the standard deviation sigma_e

#compute the variance of the normal distribution from which the estimated weights come from
var_w = np.array(np.diag(XX_inv) * sigma2_hat).reshape(2,1)

#compute the confidence intervals (using the t-student distribution, assuming sigma_e unknown)
CI_min = w_hat - t.ppf(1-0.025,m-d-1)*np.sqrt(var_w) #alpha=0.05 
CI_max = w_hat + t.ppf(1-0.025,m-d-1)*np.sqrt(var_w) #alpha=0.05 
print("\nConfidence intervals for the two weights:\n",np.hstack((CI_min,CI_max)))

Estimated variance of noise:
 [0.01146231]

Confidence intervals for the two weights:
 [[ 2.97233295  3.0204949 ]
 [-0.0163543   0.01909066]]
